In [35]:
from sklearn.model_selection import train_test_split
import pandas as pd
import sklearn
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score,RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from numpy import mean, std

In [3]:
NTS_dummy = pd.read_csv('/content/drive/MyDrive/socio_pred/UKDA-5340-tab/NTS_v2.csv')
# replace NA with zero
#na occurs for std of users with one trip, or count of users with no weekend trip
NTS_dummy = NTS_dummy.fillna(0)

mobility = NTS_dummy.iloc[:,38:]
socio = NTS_dummy.drop(columns = ['IndividualID','SurveyYear']).iloc[:,0:36]

socio_cols = {'age' : socio.columns[0:6],
'sex' : socio.columns[6:8],
'marital' : socio.columns[8:10],
'income': socio.columns[10:13],
'work': socio.columns[13:16],
'hhincome': socio.columns[16:19],
'hh_struc': socio.columns[19:25],
'hh_work': socio.columns[25:28],
'hh_socialclass': socio.columns[28:34],
'urban_rural': socio.columns[34:36]}

In [18]:
# Train Test split
mobility = NTS_dummy.iloc[:,38:]
socio = NTS_dummy.drop(columns = ['IndividualID','SurveyYear']).iloc[:,0:36]
X_train, X_test, y_all_tr, y_all_ts = train_test_split(mobility, socio, random_state=42)



In [37]:
#normalize
scaler = MinMaxScaler().fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train),columns = X_train.columns)
#PCA
pca = PCA(n_components=0.85) #85% of variance explained
PC_train = pca.fit_transform(X_train)

#transoform test set
X_test = pd.DataFrame(scaler.transform(X_test),columns = X_test.columns)
PC_test = pca.transform(X_test)


#principalDf = pd.DataFrame(data = principalComponents)
pca.n_components_


10

In [40]:
#select dependant var
def dependant(train,test,cols,var):
  y_train = train[cols[var]]
  y_test = test[cols[var]]

  # convert to one columns labeles
  label_encoder = LabelEncoder()
  y_train = y_train.idxmax(axis=1)
  y_test = y_test.idxmax(axis=1)
  label_encoder.fit(y_train)
  y_train = label_encoder.transform(y_train)
  y_test = label_encoder.transform(y_test)



  return y_train,y_test,label_encoder

#Dependant variable: Age category

In [42]:
y_train,y_test, le = dependant(y_all_tr,y_all_ts,socio_cols,'age')
#to get back labels
#le.inverse_transform(y_train)

## RF

In [43]:
#feature_names = mobility.iloc[:,:-1].columns
forest_age = RandomForestClassifier(n_estimators=200, criterion='gini',\
                                    min_samples_split=5, min_samples_leaf=2,\
                                    max_features='auto', bootstrap=True,\
                                    n_jobs=-1, random_state=42)

forest_age.fit(PC_train,y_train)
y_pred = forest_age.predict(PC_test)

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred))
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred))
# # define the model evaluation procedure
# cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# # evaluate the model and collect the scores
# n_scores = cross_val_score(forest, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
# # report the model performance
# print('Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))




[[  0   0   0  11   1   0]
 [  0   0   5  47   1   0]
 [  0   0   7  88  16   1]
 [  0   0   3  87  15   1]
 [  0   0  11  98  14   6]
 [  0   0  18 143  26  13]]
0.1977124183006536


In [ ]:
from sklearn.model_selection import GridSearchCV

# Create the parameter grid based on the results of random search 
param_grid = {
    'max_depth': [10,15],
    'max_features': [2,4],
    'min_samples_leaf': [1, 5],
    'min_samples_split': [2,5],
    'n_estimators': [100, 200, 500]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 8, n_jobs = -1, verbose = 5)

# Fit the grid search to the data
grid_search.fit(PC_train,y_train)

Fitting 8 folds for each of 384 candidates, totalling 3072 fits


In [ ]:
samp = socio[income]
samp['TripEnd_B01ID_9'] = mobility['TripEnd_B01ID_9'] #most important
samp['TripStart_B01ID_4'] = mobility['TripStart_B01ID_4'] #least important

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#sample of the most important factor for predictin income
#avg number of trips for lower income
samp[samp['IndIncome2002_B02ID_1']==1]['TripEnd_B01ID_9'].mean()

1.3994252873563218

In [ ]:
#avg number of trips for middle income

samp[samp['IndIncome2002_B02ID_2']==1]['TripEnd_B01ID_9'].mean()

1.6370967741935485

In [ ]:
#avg number of trips for higher income
samp[samp['IndIncome2002_B02ID_3']==1]['TripEnd_B01ID_9'].mean()

1.7831715210355987

In [ ]:
#sample of the least important factor for predicting income
#avg number of trips for lower income

samp[samp['IndIncome2002_B02ID_1']==1]['TripStart_B01ID_4'].mean()

0.004310344827586207

In [ ]:
#avg number of trips for middle income

samp[samp['IndIncome2002_B02ID_2']==1]['TripStart_B01ID_4'].mean()

0.004032258064516129

In [ ]:
#avg number of trips for higher income
samp[samp['IndIncome2002_B02ID_3']==1]['TripStart_B01ID_4'].mean()

0.003236245954692557

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
# define the model evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(model, x, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report the model performance
print('Mean Accuracy: %.3f' % (mean(n_scores)))

Mean Accuracy: 0.579
